# 유사한 단어 찾기 게임

1. 사전 학습된 모델 또는 적절한 데이터셋을 찾는다.
2. 워드 임베딩 모델을 학습시킨다.
3. 단어 유사도가 0.8 이상인 A, B를 랜덤 추출한다.
4. A, B와 대응되는 C를 추출한다.
5. D를 입력받는다.

=>
A:B = C:D 관계에 대응하는 D를 찾는 게임을 만든다. 
ex) A: 산, B: 바다, C: 나무, D: 물

**<출력 예시>**
- 관계 [수긍 : 추락 = 대사관 : ?]
- 모델이 예측한 가장 적합한 단어: 잠입
- 당신의 답변과 모델 예측의 유사도: 0.34
- 아쉽네요. 더 생각해보세요.

In [1]:
import random
import re
from gensim.models.fasttext import load_facebook_vectors

In [2]:
model_path = 'cc.ko.300.bin'
model = load_facebook_vectors(model_path)

In [9]:
_HANGUL_PATTERN = re.compile(r"^[가-힣]{2,6}$")

def get_korean_vocab(model, limit):
    vocab = list(model.key_to_index.keys())[:limit]
    korean_vocab = set()

    for word in vocab:
        # 정규식으로 2~6글자의 한글 단어만 필터링 (조사, 특수문자 등 제외)
        if _HANGUL_PATTERN.fullmatch(word):
            korean_vocab.add(word)
            
    return list(korean_vocab)

def generate_analogy_quiz(vocab, model, min_similarity=0.8):
    while True:
        try:
            # 1. 랜덤 단어 A를 선택
            a = random.choice(vocab)
            
            # 2. A와 유사도 0.8 이상인 단어 B를 찾음 (상위 5개 후보 중)
            similar_words = model.most_similar(a, topn=5)
            
            # 적절한 B 후보 찾기
            b_candidates = [
                word for word, sim in similar_words 
                if sim >= min_similarity and _HANGUL_PATTERN.fullmatch(word) and word != a
            ]
            
            if not b_candidates:
                continue
            
            b = random.choice(b_candidates)

            # 3. A, B와는 다른 C를 랜덤하게 선택
            for _ in range(50):         
                c = random.choice(vocab)
                if c != a and c != b and model.similarity(a, c) < 0.6 and model.similarity(b, c) < 0.6:
                    break
            else:
                continue

            # 4. 모델을 이용해 정답 D를 예측
            predicted_results = model.most_similar(positive=[b, c], negative=[a], topn=5)
            
            # 5. 예측된 D가 문제로 적합한지 검증
            for d_predicted, _ in predicted_results:
                if (_HANGUL_PATTERN.fullmatch(d_predicted) and 
                    d_predicted not in [a, b, c]):
                    return a, b, c, d_predicted
                    
        except KeyError:
            continue

In [10]:
def play_game(vocab, model):
    # 1. 유효한 유추 문제 생성
    A, B, C, predicted_D = generate_analogy_quiz(vocab, model)

    # 2. 사용자에게 문제 제시
    print("\n" + "="*30)
    print(f"다음 관계를 보고 '?'에 들어갈 단어를 맞춰보세요.\n")
    print(f"관계:  [{A} : {B}  =  {C} : ?]")
    print("="*30)
    
    # 3. 사용자 답변 입력
    user_answer = input("당신의 답변: ").strip()

    # 4. 결과 출력
    print("\n--- 결과 ---")
    print(f"모델이 예측한 가장 적합한 단어: {predicted_D}")
    print(f"당신의 답변: {user_answer}")

    if user_answer == predicted_D:
        print("\n🎉 정답입니다! 모델의 예측과 정확히 일치해요.")
        return

    try:
        # 사용자가 입력한 단어가 모델 어휘에 있는지 확인
        if user_answer in model:
            similarity = model.similarity(user_answer, predicted_D)
            print(f"당신의 답변과 모델 예측의 유사도: {similarity:.2f}")
            
            if similarity > 0.7:
                print("👍 정답에 매우 가까워요! 좋은 답변입니다.")
            elif similarity > 0.4:
                print("🤔 아쉽네요. 더 생각해보세요.")
            else:
                print("❌ 많이 벗어난 것 같아요. 다른 단어를 떠올려 보세요.")
        else:
            print("입력한 단어는 모델의 어휘 사전에 없어요!")
            
    except KeyError:
         print("입력한 단어는 모델의 어휘 사전에 없어요!")

In [11]:
korean_vocab = get_korean_vocab(model, limit=50000)

In [19]:
# 게임 시작
play_game(korean_vocab, model)


다음 관계를 보고 '?'에 들어갈 단어를 맞춰보세요.

관계:  [이르되 : 가로되  =  운동하는 : ?]

--- 결과 ---
모델이 예측한 가장 적합한 단어: 운동하고
당신의 답변: 운동하며
당신의 답변과 모델 예측의 유사도: 0.52
🤔 아쉽네요. 더 생각해보세요.
